# Libraries / ライブラリー

In [1]:
# Install packages
# パッケージのインストール
!pip install pydeck-carto geopandas carto-auth[carto-dw] db_dtypes -q xgboost category-encoders numpy shap matplotlib seaborn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.8/81.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.9/547.9 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 55.4 MB/s eta 0:00:00


In [2]:
# Carto / GoogleBigQuery
import pydeck as pdk
import pydeck_carto as pdkc
from carto_auth import CartoAuth
from google.cloud import bigquery

# Plotting / プロット
import matplotlib.pyplot as plt
import seaborn as sns
import shap

# Analysis / 分析
import numpy as np
import pandas as pd
import geopandas as gpd

# Machine Learning / 機械学習
import xgboost as xgb
import category_encoders as ce
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import (
    accuracy_score,
    mean_absolute_error,
    mean_squared_error,
    r2_score,
)
from sklearn.model_selection import (
    GridSearchCV,
    KFold,
    LeaveOneOut,
    cross_val_score,
    train_test_split,
)

Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)


# Authentication / 認証

In [3]:
# Authenticate Carto
# Cartoを認証する
carto_auth = CartoAuth.from_oauth()
carto_dw_client = carto_auth.get_carto_dw_client()

Go to the following URL:
https://auth.carto.com/authorize?client_id=0dxb8HR3ATXCxJiPOJVHsLoHoAtbRX6u&response_type=code&scope=None&audience=carto-cloud-native-api&redirect_uri=https%3A%2F%2Fapp.carto.com%2Fauth%2Ftoken&code_challenge_method=S256&code_challenge=7pP6BOJfHvtZeQlICj3OiLGIN3D78ecvBTxn6mGibrI
Enter the Access code: 
8MSgABBCH69MVEQKBFmlqNRY1-6OWGgDbwvPW7MkOxuUd


# Accessing Data / データへのアクセス

## CARTO Data Warehouse / CARTOデータウェアハウス

In [5]:
# List datasets in Carto Data Warehouse
# Cartoデータウェアハウスのデータセットの表示
datasets = list(carto_dw_client.list_datasets())

if datasets:
    print("Datasets in CARTO Data Warehouse:")
    for dataset in datasets:
        print("\t{}".format(dataset.dataset_id))
else:
    print("CARTO Data Warehouse project does not contain any datasets.")

Datasets in CARTO Data Warehouse:
	private_christopher_kyed_3_a1907fcc
	shared
	shared_us
	workflows_temp_christopher_kyed_3_a1907fcc
	workflows_temp_christopher_kyed_3_a1907fcc_us


In [ ]:
# List tables in chosen dataset
# 選択したデータセットのテーブルをリストアップする
dataset_id = 'shared_us'
tables = carto_dw_client.list_tables(dataset_id)  # Make an API request.

print("Tables contained in '{}':".format(dataset_id))
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))


In [ ]:
# Provide your own Carto Data Observatory ID here.
# ここにあなた自身のCarto Data Observatory IDを入力してください。
cdw_dataset = "carto-dw-ac-zn8l36fh"

In [ ]:
# Query the London Listings table and visualize as GeoDataFrame
# London Listingsテーブルを照会し、GeoDataFrameとして可視化する。
table = carto_dw_client.get_table(f"{cdw_dataset}.shared_us.london_listings_2023")
gdf = carto_dw_client.list_rows(table).to_geodataframe(
    geography_column="geom", create_bqstorage_client=False
)
display(gdf.head())

## Data Observatory / データ・オブザーバートリー

In [ ]:
# Provide your own Carto Data Observatory ID here.
# ここにあなた自身のCarto Data Observatory IDを入力してください。
dataobs_dataset = "carto-data.ac_zn8l36fh"

In [ ]:
# Get a list of all subscribed tables
# 購読されている全テーブルのリストを取得する
get_subscriptions_q = f"""
  CALL `carto-un`.carto.DATAOBS_SUBSCRIPTIONS('{dataobs_dataset}',"dataset_license = 'Public data'");
  """

subs_df = (
    carto_dw_client.query(get_subscriptions_q)
    .result()
    .to_dataframe(create_bqstorage_client=False)
)
display(subs_df)

In [ ]:
# Get the variables of a specific dataset
# 特定のデータセットの変数を取得する
get_dataset_variables = f"""
CALL `carto-un`.carto.DATAOBS_SUBSCRIPTION_VARIABLES(
    "{dataobs_dataset}",
    "dataset_slug = 'ons_ltla_a64e5794'");
"""
carto_dw_client.query(get_dataset_variables).result().to_dataframe(create_bqstorage_client=False)

In [10]:
# Extract table ID and associated geography table if present
# テーブルIDと関連する地理テーブルがあればそれを抽出する。
table_id, geography_id = subs_df.query("dataset_slug == 'ons_ltla_a64e5794'")[
    ["dataset_table", "associated_geography_table"]
].values.ravel()
print(table_id, geography_id)


sub_gbr_ons_geography_gbr_ltla_2021 None


In [ ]:
# Query table and visualzie as GeoDataFrame
# GeoDataFrameとしてテーブルとビジュアルをクエリーする
london_auth_q = f"""
  SELECT *
  FROM `{dataobs_dataset}.{dataset_id}`
"""
london_auth = (
    carto_dw_client.query(london_auth_q)
    .result()
    .to_dataframe(create_bqstorage_client=False)
)
london_auth.sample(5)


# Visualizing Data / データの可視化

## CARTO Data Warehouse / CARTOデータウェアハウス

In [ ]:
# Register CartoLayer in pydeck
# pydeckにCartoLayerを登録する。
pdkc.register_carto_layer()

# Render first CartoLayer CartoLayer
# 最初の CartoLayer CartoLayer をレンダリングする。
layer = pdk.Layer(
    "CartoLayer",
    data=f"SELECT geom, price FROM `{cdw_dataset}.shared_us.london_listings_2023_joined`",
    type_=pdkc.MapType.QUERY,
    connection=pdkc.CartoConnection.CARTO_DW,
    credentials=pdkc.get_layer_credentials(carto_auth),
    point_radius_min_pixels=2.5,
    get_fill_color=pdkc.styles.color_bins(
        "price", [0, 100, 200, 300, 400, 500], "PinkYl"
    ),
    get_line_color=[0, 0, 0, 100],
)

# Set initial viewing location/zoom/angle
# 初期表示位置/ズーム/アングルを設定する
view_state = pdk.ViewState(
    latitude=51.50071697877869, longitude=-0.12461158468895285, zoom=8
)

# Initialize map
# マップの初期化
pdk.Deck(layer, map_style=pdk.map_styles.ROAD, initial_view_state=view_state).to_html(
    iframe_height=800
)


In [ ]:
# Register CartoLayer in pydeck
# pydeckにCartoLayerを登録する。
pdkc.register_carto_layer()

# Render first CartoLayer CartoLayer
# 最初の CartoLayer CartoLayer をレンダリングする。
layer = pdk.Layer(
    "CartoLayer",
    data=f"SELECT h3, price FROM `{cdw_dataset}.shared_us.london_listings_2023_joined`",
    type_=pdkc.MapType.QUERY,
    connection=pdkc.CartoConnection.CARTO_DW,
    credentials=pdkc.get_layer_credentials(carto_auth),
    aggregation_exp=pdk.types.String("avg(price) as price"),
    aggregation_res_level=8,
    geo_column=pdk.types.String("h3"),
    get_fill_color=pdkc.styles.color_bins(
        "price", [0, 100, 200, 300, 400, 500], "PurpOr"
    ),
    get_line_color=[0, 0, 0, 100],
    line_width_min_pixels=1,
    opacity=0.4,
    stroked=True,
    extruded=False,
    pickable=True,
)

# Add tooltip to show values on map
# 地図上に値を表示するツールチップを追加する
tooltip = {
    "html": "H3: <b>{id}</b><br />Price: <b>{price}</b>",
    "style": {
        "background": "grey",
        "color": "white",
        "font-family": '"Helvetica Neue", Arial',
        "z-index": "10000",
    },
}

# Set initial viewing location/zoom/angle
# 初期表示位置/ズーム/アングルを設定する
view_state = pdk.ViewState(
    latitude=51.50071697877869, longitude=-0.12461158468895285, zoom=8
)

# Initialize map
# マップの初期化
pdk.Deck(
    layer, map_style=pdk.map_styles.ROAD, tooltip=tooltip, initial_view_state=view_state
).to_html(iframe_height=800)

## Data Observatory / / データ・オブザーバートリー

In [ ]:
# Get a list of all subscribed tables again
# 再びすべての購読テーブルのリストを取得する
get_subscriptions_q = f"""
  CALL `carto-un`.carto.DATAOBS_SUBSCRIPTIONS('{dataobs_dataset}',"dataset_license = 'Public data'");
  """

subs_df = carto_dw_client.query(get_subscriptions_q).result().to_dataframe(create_bqstorage_client=False)
display(subs_df)

In [15]:
# Extract table ID and associated geography table if present
# テーブルIDと関連する地理テーブルがあればそれを抽出する。
dataset_id, geography_id = subs_df.query(
    "dataset_name == 'Lower Tier Local Authority - United Kingdom (2021)'"
)[["dataset_table", "associated_geography_table"]].values.ravel()

In [ ]:
# Register CartoLayer in pydeck
# pydeckにCartoLayerを登録する。
pdkc.register_carto_layer()


# Render first CartoLayer CartoLayer
# 最初の CartoLayer CartoLayer をレンダリングする。
layer = pdk.Layer(
    "CartoLayer",
    data = f"SELECT * FROM `{dataobs_dataset}.{dataset_id}`",
    geo_column=pdk.types.String("geom"),
    type_=pdkc.MapType.QUERY,
    connection=pdkc.CartoConnection.CARTO_DW,
    credentials=credentials=pdkc.get_layer_credentials(carto_auth),
    get_fill_color=[238, 77, 90],
    get_line_color=[0, 0, 0, 100],
    line_width_min_pixels=1,
    opacity=0.2,
    pickable=True,
    stroked=True,
    )

# Add tooltip to show values on map
# 地図上に値を表示するツールチップを追加する
tooltip = {
    "html": "Mobility: <b>{y2016}</b>",
    "style": {
        "background": "grey",
        "color": "white",
        "font-family":
        '"Helvetica Neue", Arial',
        "z-index": "10000"
    },
}

# Set initial viewing location/zoom/angle
# 初期表示位置/ズーム/アングルを設定する
view_state = pdk.ViewState(
    latitude=51.50071697877869,
    longitude=-0.12461158468895285,
    zoom=5
)

# Initialize map
# マップの初期化
pdk.Deck(
    layer,
    tooltip = tooltip,
    initial_view_state=view_state,
    map_style=pdk.map_styles.LIGHT_NO_LABELS
).to_html(iframe_height = 800)

In [17]:
# Extract table ID and associated geography table if present
# テーブルIDと関連する地理テーブルがあればそれを抽出する。
dataset_id, geography_id = subs_df.query("dataset_name == 'Residential Mobility Index - United Kingdom (Lower Super Output Area)'")[["dataset_table", "associated_geography_table"]].values.ravel()

In [ ]:
# Register CartoLayer in pydeck
# pydeckにCartoLayerを登録する。
pdkc.register_carto_layer()

# Render first CartoLayer CartoLayer
# 最初の CartoLayer CartoLayer をレンダリングする。
layer_one = pdk.Layer(
    "CartoLayer",
    data = f"""SELECT y2016, geom
    FROM `{dataobs_dataset}.{dataset_id}` d
    JOIN `{dataobs_dataset}.{geography_id}` g
    ON d.geoid = g.geoid""",
    geo_column=pdk.types.String("geom"),
    type_=pdkc.MapType.QUERY,
    connection=pdkc.CartoConnection.CARTO_DW,
    credentials=credentials=pdkc.get_layer_credentials(carto_auth),
    get_fill_color=pdkc.styles.color_bins("y2016",[0, 0.07, 0.08, 0.09, 0.1, 0.11, 1], "Emrld"),
    get_line_color=[0, 0, 0, 100],
    line_width_min_pixels=1,
    opacity=0.5,
    pickable=True,
    stroked=True,
    )

# Render first CartoLayer CartoLayer
# 2番目のCartoLayer CartoLayer をレンダリングする。
layer_two = pdk.Layer(
    "CartoLayer",
    data=f"SELECT geom, price FROM `{cdw_dataset}.shared_us.london_listings_2023_joined`",
    type_=pdkc.MapType.QUERY,
    connection=pdkc.CartoConnection.CARTO_DW,
    credentials=pdkc.get_layer_credentials(carto_auth),
    point_radius_min_pixels=2.0,
    get_fill_color=[255, 100, 100],
    get_line_color=[0, 0, 0, 100]
)

# Add tooltip to show values on map
# 地図上に値を表示するツールチップを追加する
tooltip = {
    "html": "Mobility: <b>{y2016}</b>",
    "style": {
        "background": "grey",
        "color": "white",
        "font-family":
        '"Helvetica Neue", Arial',
        "z-index": "10000"
    },
}

# Set initial viewing location/zoom/angle
# 初期表示位置/ズーム/アングルを設定する
view_state = pdk.ViewState(
    latitude=51.50071697877869, longitude=-0.12461158468895285, zoom=8
)

# Initialize map
# マップの初期化
pdk.Deck(
    [layer_one, layer_two],
    tooltip = tooltip,
    initial_view_state=view_state,
    map_style=pdk.map_styles.LIGHT,
).to_html(iframe_height = 800)

# Data preparation / データ準備

## Data cleaning / データクリーニング

In [ ]:
# Load joined Carto Workflow table into a DataFrame
# 結合されたCarto WorkflowテーブルをDataFrameにロードする。
table = carto_dw_client.get_table(
    f"{cdw_dataset}.shared_us.london_listings_2023_joined"
)
gdf_load = carto_dw_client.list_rows(table).to_geodataframe(
    geography_column="geom", create_bqstorage_client=False
)

# Create an index on the table
# テーブルにインデックスを作成する
idx = "index"
gdf_load = gdf_load.reset_index()
gdf_load = gdf_load.set_index(idx)
gdf_load


In [20]:
# Drop geometry columns
# ジオメトリの列を削除する
gdf_clean = gdf_load.drop(['h3_geo', 'h3', 'geom', 'h3_joined'], axis=1)

In [ ]:
# Rename columns created by join in workflow
# ワークフローで結合によって作成された列の名前を変更する
gdf_clean.rename(columns={"population_joined": "population",
                          "leisure_joined": "lesiure",
                          "tourism_joined": "tourism",
                          "transportation_joined": "transportation"}, inplace=True)
gdf_clean

In [22]:
# Declare numerical, categorical and target variables
# 数値変数、カテゴリー変数、ターゲット変数の宣言
categorical_variables = ["neighbourhood", "room_type", "workplace_zone"]
numerical_variables = [
    "minimum_nights",
    "number_of_reviews",
    "reviews_per_month",
    "availability_365",
    "population",
    "lesiure",
    "tourism",
    "transportation",
    "mobility",
]
target = "price"

In [ ]:
# Check data types for each column
# 各列のデータ型をチェックする
gdf_clean.dtypes

In [ ]:
# Force data types for our variables
# 変数のデータ型を強制する
for cat in categorical_variables:
  gdf_clean[cat] = gdf_clean[cat].astype('object')

for num in [numerical_variables + [target]]:
  gdf_clean[num] = gdf_clean[num].astype('float64')

gdf_clean

## Feature Analysis / 特徴分析

In [ ]:
# Check correlation between input variables
# 入力変数間の相関をチェックする
correlations = gdf_clean.corr(numeric_only=True)

fig, axes = plt.subplots(figsize=(10,10), facecolor='white')
sns.heatmap(correlations, vmax=1.0, center=0, fmt='.2f', cmap="YlGnBu",
            square=True, linewidths=.5, annot=True, cbar_kws={"shrink": .70}, ax=axes)
plt.show()

In [ ]:
# Count plot for target variable
# ターゲット変数のカウントプロット
sns.set(style='whitegrid', palette="deep", font_scale=1.1, rc={"figure.figsize": [10, 10]})
sns.displot(gdf_clean['price'], kde=False, bins=20,).set(xlabel='Price', ylabel='Count');

In [ ]:
# Count plot for target numerical variables
# 数値変数のカウントプロット
gdf_clean[numerical_variables].hist(bins=15, figsize=(15, 6), layout=(2, 5));

In [ ]:
# Count plot for target categorical variables
# カテゴリー変数のカウントプロット

# Select the columns to be plotted
# プロットする列を選択する
cols = ["room_type", "workplace_zone"]

fig, axes = plt.subplots(1, 2)
axes = axes.ravel()

for col, ax in zip(cols, axes):
    sns.histplot(gdf_clean[col], ax=ax).set_xticklabels(
        ax.get_xticklabels(), rotation=90
    )

fig.tight_layout()
plt.show()

## Null values and Outliers

In [ ]:
# Use df.describe to check percentiles for each numerical colummn
# df.describeを使って各数値列のパーセンタイルをチェックする
gdf_selected = gdf_clean.copy()
gdf_selected.describe(percentiles=[.25, .5, .75, .95, .99])

In [ ]:
# Remove data from selected columns not within the 99th percentile
# 選択された列から99パーセンタイル以内のデータを削除する。
gdf_selected = gdf_selected[gdf_selected['price'] >= 10]
gdf_selected = gdf_selected[gdf_selected['price'] <= 1570]
gdf_selected = gdf_selected[gdf_selected['minimum_nights'] <= 90]
gdf_selected

In [ ]:
# Display null values
# ヌル値を表示する
display(gdf_selected.isnull().sum())

In [ ]:
# Fill remaining null values with 0
# 残りのヌル値を0で埋める
gdf_selected = gdf_selected.fillna(0)
display(gdf_selected.isnull().sum())

## Encoding

In [33]:
gdf_encoded = gdf_selected.copy()

In [ ]:
# Display categorical features and their respective cardinality
# カテゴリカルな特徴とそれぞれのカーディナリティを表示します
display(gdf_encoded[categorical_variables])
for cat in categorical_variables:
    print(cat, gdf_encoded[cat].nunique())

In [ ]:
# Nominal variable and cardinality < 15, use OneHot Encoding
# 名目変数でカーディナリティが15未満の場合、OneHotエンコーディングを使用する。
gdf_encoded = pd.get_dummies(gdf_encoded, columns=["room_type", "workplace_zone"])

# Nominal variable and cardinality > 15, use Target Encoding
# 名目変数でカーディナリティが15を超える場合は、ターゲットエンコーディングを使用する。
encoder = ce.TargetEncoder()
gdf_encoded["neighbourhood"] = encoder.fit_transform(gdf_encoded['neighbourhood'], gdf_encoded['price'])
gdf_encoded

In [ ]:
# Check dtypes of final cleaned,selected and encoded table
# 最終的にクリーニングされ、選択され、エンコードされたテーブルのd型をチェックする
gdf_encoded.dtypes

In [37]:
# Split dataset into dependent and independent variables (X and y)
# データセットを従属変数と独立変数 (X と y) に分割する
X = gdf_encoded.drop("price", axis=1).copy()
y = gdf_encoded["price"].copy()

# Split X and y data into train and test data 80/20
# XとYのデータを訓練データとテストデータに分ける 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14, test_size=.2)

In [ ]:
# Visualize final split data
# 最終的な分割データを視覚化する
display(X_train)
display(y_train)
display(X_test)
display(y_test)

# Models

## XGBOOST

In [46]:
# Create model instance
# モデルのインスタンスを作成する
xgb_model = xgb.XGBRegressor(
    seed=39,
    eval_metric=["mae", "rmse"],
    gpu_id=0,
    tree_method = 'gpu_hist',
    predictor = 'gpu_predictor'
)

In [47]:
# Train the model
# モデルをトレーニングする
eval_set = [(X_train, y_train), (X_test, y_test)]
xgb_model.fit(X_train, y_train, verbose=0, eval_set=eval_set)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=['mae', 'rmse'],
             feature_types=None, gamma=None, gpu_id=0, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, n_estimators=100, n_jobs=None,
             num_parallel_tree=None, predictor='gpu_predictor',
             random_state=None, ...)

In [ ]:
# Make predictions on our test data
# テストデータで予測を行う
preds = xgb_model.predict(X_test)
predictions = [round(value) for value in preds]
print(predictions[0:10])
print([round(value) for value in list(y_test)][0:10])

In [ ]:
# Evaluate model
# モデルを評価する

# R2 score / R2スコア
print("R^2 :", r2_score(y_test, predictions))
# Adjusted R2 / 調整後R2
print("Adjusted R^2 :", 1 - (1-xgb_model.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1))
# Mean Average Error / 平均平均誤差
print("MAE :", mean_absolute_error(y_test,predictions))
# Root Mean Squared Error / 平均二乗誤差
print("RMSE:", np.sqrt(mean_squared_error(y_test, predictions)))

## Validation & Plotting

In [51]:
# Calculate the shap values (GPU reccomended)
# シャップ値を計算する（GPU推奨）
explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test)
expected_value = explainer.expected_value

if isinstance(expected_value, list):
    expected_value = expected_value[1]
print(f"Explainer expected value: {expected_value}")

Explainer expected value: 167.58731079101562


In [ ]:
# Create summary plot, showing the most impactful features
# 最もインパクトのある特徴を示すサマリープロットを作成する
shap.summary_plot(shap_values, X_test, plot_type="bar", show=True)

In [ ]:
# Same plot as above but this time we can see the distribution of values
# 上と同じプロットだが、今回は値の分布を見ることができる
shap.summary_plot(shap_values, X_test, plot_type='violin', show=True)

In [ ]:
# Create a decision plot for the first 5 predictions, showing how the model arrived at its prediction
# 最初の5つの予測についてデシジョンプロットを作成し、モデルがどのように予測に至ったかを示す
select = range(5)
features = X_test.iloc[select]
shap_values = explainer.shap_values(features)

shap.decision_plot(expected_value, shap_values, features, show=True)

In [ ]:
# Same plot as above but with one prediction only so we can see the values at each step
# 各ステップでの値を見ることができるように、上記と同じプロットだが、予測は1回のみとする
select = range(1)
features = X_test.iloc[select]

shap_values = explainer.shap_values(features)
shap.decision_plot(expected_value, shap_values, features, show=True)

In [ ]:
# Create force plot for the same predictions, a different way of visualizing feature impact
# 同じ予測に対してフォースプロットを作成し、特徴の影響を別の方法で視覚化する
select = range(1)
features = X_test.iloc[select]
shap_values = explainer.shap_values(features)

shap.force_plot(expected_value, shap_values, features, text_rotation=10, matplotlib=True)

## Model optimization


In [ ]:
# Initialize default model and choose the combination of hyperparameters to use during GridSearch
# デフォルトモデルを初期化し、グリッドサーチ時に使用するハイパーパラメータの組み合わせを選択する
reg_model = xgb.XGBRegressor(gpu_id=0, tree_method='gpu_hist', predictor='gpu_predictor')
grid = {
    "learning_rate": [0.05, 0.1],
    "max_depth": [8, 10, 12],
    "min_child_weight": [1, 3],
    "gamma":[0.0, 0.1],
    "colsample_bytree":[0.5, 1]
}

# Run the GridSearch (around 2 minutes, GPU session reccomended)
# グリッドサーチの実行（2分程度、GPUセッション推奨）
grid_search = GridSearchCV(
            estimator=reg_model,
            param_grid=grid,
            cv=2,
            scoring='r2', verbose=1, n_jobs=-1)
grid_search.fit(X_train, y_train)

In [ ]:
# Print the best parameters found by GridSearch
# GridSearchによって見つかった最適なパラメータを表示する
print(f"Best parameters: {grid_search.best_params_}")

In [ ]:
# Print a few of the many runs made by during the GridSearch
# GridSearch中に実行された多くのランのうちのいくつかを表示する
pd.DataFrame(grid_search.cv_results_).head()

In [ ]:
# Create optimzed model instance
# 最適化されたモデルインスタンスを作成する
xgb_model_opt = xgb.XGBRegressor(
    **grid_search.best_params_, # Best parameters / 最高のパラメーター
    n_estimators = 200, # More complex model so use twice the estimators / より複雑なモデルのため、2倍の推定量を使用
    seed=39,
    eval_metric=["mae", "rmse"],
    gpu_id=0,
    tree_method = 'gpu_hist',
    predictor = 'gpu_predictor'
)

# Train the model
# モデルをトレーニングする
eval_set = [(X_train, y_train), (X_test, y_test)]
xgb_model_opt.fit(X_train, y_train, verbose=0, eval_set=eval_set)

# Make predictions on our test data
# テストデータで予測を行う
preds_opt = xgb_model_opt.predict(X_test)
predictions_opt = [round(value) for value in preds_opt]

# R2 score / R2スコア
print("R^2 :", r2_score(y_test, predictions))
# Adjusted R2 / 調整後R2
print("Adjusted R^2 :", 1 - (1-xgb_model.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1))
# Mean Average Error / 平均平均誤差
print("MAE :", mean_absolute_error(y_test,predictions))
# Root Mean Squared Error / 平均二乗誤差
print("RMSE:", np.sqrt(mean_squared_error(y_test, predictions)))

# Prediction

In [ ]:
# Make predictions and joined them back to test data
# 予測を立て、それをテストデータに結びつける
gdf_out = X_test.copy()
gdf_out["price_predicted"] = xgb_model_opt.predict(X_test)
gdf_out = gdf_out.merge(gdf_load, left_on="index", right_on="index")[
    ["h3", "price", "price_predicted"]
]
gdf_out.reset_index(drop=True, inplace=True)
gdf_out

In [62]:
# Export DataFrame to local file (.parquet)
# データフレームをローカルファイル（.parquet）にエクスポートする。
# gdf_out.to_parquet('london_listings_2023_predicted.parquet')

In [ ]:
# Upload result to our Carto account and overwrite same table if already present
# 結果をCartoアカウントにアップロードし、同じテーブルが存在する場合は上書きする
job_config = bigquery.LoadJobConfig(schema=[], write_disposition="WRITE_TRUNCATE")
carto_dw_client.load_table_from_dataframe(
    gdf_out,
    f"{cdw_dataset}.shared_us.london_listings_2023_predicted",
    job_config=job_config,
).result()

In [ ]:
# List tables again and see if out new table is there
# テーブルを再度リストアップし、新しいテーブルがあるかどうかを確認する
dataset_id = 'shared_us'
tables = carto_dw_client.list_tables(dataset_id)

print("Tables contained in '{}':".format(dataset_id))
for table in tables:
    print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

In [ ]:
# Create final map to visualize our predictions
# 予測を視覚化する最終マップを作成する
pdkc.register_carto_layer()

layer = pdk.Layer(
    "CartoLayer",
    data=f"SELECT h3, price_predicted, price FROM `{cdw_dataset}.shared_us.london_listings_2023_predicted`",
    type_=pdkc.MapType.QUERY,
    connection=pdkc.CartoConnection.CARTO_DW,
    credentials=pdkc.get_layer_credentials(carto_auth),

    aggregation_exp=pdk.types.String("avg(price_predicted) as price_predicted"),
    aggregation_res_level=8,
    geo_column=pdk.types.String("h3"),

    #get_fill_color='[255, 255, properties.growth * 255]'
    get_fill_color=pdkc.styles.color_bins("price_predicted",[0, 100, 200, 300, 400, 500], "Sunset"),
    opacity=0.5,

    get_elevation='properties.price_predicted * properties.price_predicted',
    elevation_scale=0.01,

    get_elevation='properties.price_predicted',
    elevation_scale=10,

    # get_line_color=[0, 0, 0, 100],
    # line_width_min_pixels=1,

    stroked=False,
    filled=True,
    extruded=True,
    wireframe=True,
    auto_highlight=True,
    pickable=True
)

tooltip = {"html": "<b>H3 Index:</b> {id} <br /><b>Predicted Price:</b> {price_predicted}"}

view_state = pdk.ViewState(
    latitude=51.50071697877869,
    longitude=-0.12461158468895285,
    zoom=8,
    pitch=45,
    bearing=0
  )

pdk.Deck(layer, map_style=pdk.map_styles.DARK, tooltip=tooltip, initial_view_state=view_state).to_html(iframe_height=800)